# Capstone Regression Project
### Scharmaine Chappell

## Business Understanding
My stakeholder, One Call Concepts, Inc. is wanting to prepare for the next busy season. One Call Concepts, known by many different names, including DigSafe, is ran as Washington Utility Notification Center in Washington. (http://www.callbeforeyoudig.org/washington/faq.asp#q1) They are the middle man when a contractor, or homeowner, or anyone, wants to dig and the underground facility locating companies. OCC maintains databases of said underground facilities and uses that information to notify owners of these facilities locators of a potential dig site. When rules are followed, this allows for safer digging, lessening chances of a potential damaged line.

Working with King County, Washington, OCC has learned that the county is looking to provide incentives to new homeowners of these properties. This predicting model will allow the county to discern what type or amount of incentive to provide should they choose one of these areas. Encouraging economic growth and a more inviting, natural habitation throughout King County. Which in turn should increase interest in their county from tourists and possible new residents(constituents). Therefore, in order to encourage their working relationship, OCC would like to be able to predict the final sale prices of properties currently in areas with no view, needing beautification. Leading to increased awareness campaigns in those areas by OCC, as well as building current and new relationships with owners of underground facilities in those areas.

We will begin narrowing the variables by view. We will then remove the price outliers. From the cleaned dataset we will start with the square footage of the lot, total living area and the area above ground.

## Data Understanding
What we'll do is use data gathered on the county from 2021 - 2022 home sales data for King County Washington. https://data.kingcounty.gov/ .



## Data Preparation

### Loading the Data

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_theme(style="ticks", palette="rocket")


In [ ]:
df = pd.read_csv('data/kc_house_data.csv')

### Data Exploration

In [ ]:
#review label, types and for null values
df.info()

In [ ]:
#looking for which is carrying the most weight, mean, of the numerical catagories
df.describe()

### We see that id is the heaviest, then price, sqft_lot, sqft_living, yr_built, sqft_above. ID is the heaviest, but is not relevant for our problem, so we will drop that column first. Price will be our target. We want to know what percentage of the lot the total living space takes up. And how that takes effects the final price of the property. 

In [ ]:
df.corr()

In [ ]:
#checking for multicollinearity prior to clean up
sns.heatmap(df.corr())

In [ ]:
df.price.corr(df.sqft_lot)

In [ ]:
df.price.corr(df.sqft_living)

In [ ]:
#sqft_above is the area of the home that is above ground
df.price.corr(df.sqft_above)

### Data Cleaning

In [ ]:
#check for null values
df.isnull().sum()

#### Our heat_source and sewer_system have a negligent amount of null values in respect to the size of the size of the dataset with 32 and 14 out of 30155 entries. Therefore, we will remove these rows from our dataframe

In [ ]:
df.dropna(axis = 0, inplace = True)
#confirm null values have been removed
df.isnull().sum()

In [ ]:
#check the shape of the data verify as well
df.shape

In [ ]:
#changing the date column label to date sold to clarify what the information is representing
sold = {"date" : "datesold"}
df.rename(columns=sold, inplace=True)
df.columns

In [ ]:
#changing our datesold column from type object to type datetime 
df.datesold = df.datesold.apply(lambda x: pd.to_datetime(x, yearfirst=True))
df.dtypes

In [ ]:
#creating a new colume, 'age', from the 'yr_renovated' and 'yr_built' columns
df["age"] = np.where(df["yr_renovated"] != 0, df.datesold.apply(lambda x:x.year) - df["yr_renovated"],
df["datesold"].apply(lambda x:x.year) - df["yr_built"])
df.columns

In [ ]:
#removing current irrelevant columns
df.drop(axis = 1, labels = {'datesold','id', 'yr_renovated', 'yr_built','lat', 'long'}, inplace = True)
df.columns

In [ ]:
df.dtypes

In [ ]:
#review the address data to determine how to create a new zipcode column
df.address.tail()

In [ ]:
df.address[30111][-20:-15]

In [ ]:
df.address[30111].split(',')[2][-5:]

In [ ]:
df["zips"] = df.address.apply(lambda x: x[-20:-15])

In [ ]:
#sampling the new 'zips' column to check format
df.zips.sample(5)

In [ ]:
df.shape

In [ ]:
#now that we've separated the zip codes, we can remove the 'address' column
df.drop(axis = 1, labels = 'address', inplace = True)

In [ ]:
#verify column removed
df.shape

## Modeling

### Baseline Model

In [ ]:
#review updated dataframe
df.info()

In [ ]:
#Checking our 'view' column we see that 'NONE' is the most frequent response
#at this point we also meet our number of rows, entries, requirements
df.view.describe()

In [ ]:
#creating new dataframe with only numerical values
df_num = df[["bedrooms","bathrooms","sqft_living","sqft_lot","floors","sqft_above","sqft_basement","sqft_garage","sqft_patio","age"]]
df_num.dtypes

In [ ]:
#using just the initial numerical values to create baseline model
pred = df_num
target = df.price

In [ ]:
#assigning X and y values
X = pred
y = target

In [ ]:
baseline = sm.OLS(y, sm.add_constant(X))
results = baseline.fit()
print(results.summary())

### Our R-squared is less 40.9% using just the current numerical values as predictors and 'price' as our target. Our F-statistic and P-values are sbelow .5 as well.

### We will create some dummy variables for our catagorical columns

In [ ]:
#checking for multicollinearity
sns.heatmap(X.corr())

### We see that 'sqft_living' and 'sqft_above' (how many square feet of living space is above ground) are most correlated

In [ ]:
df.corr()

In [ ]:
df.sqft_above.corr(df.sqft_living)

### Model Iteration

In [ ]:
#reviewing data for catagorical columns
df.info()

In [ ]:
#reviewing a sample of the types of values in the catagorical values
df[["waterfront", "greenbelt", "nuisance", "view", "condition", "grade", "heat_source", "sewer_system", "zips"]].value_counts()

In [ ]:
#creating a dataframe catagorical dummy values, excluding zipcodes as we do not need them at this time 
#and they will highly skew our results
cats= ["waterfront", "greenbelt", "nuisance", "view", "condition", "grade", "heat_source", "sewer_system"]
df_dummy= pd.get_dummies(data = df, columns = cats, drop_first=True)
df_dummy.info()

In [ ]:
#removing spaces from column names and replacing with '_'
df_dummy.columns = df_dummy.columns.str.replace(' ', '_')
df_dummy.info()

In [ ]:
#creating dataframe with only rows where 'view_NONE' is True
nview_df = df_dummy[df_dummy.view_NONE == 1]
nview_df.view_NONE.value_counts()

In [ ]:
#confirming it's the entire df
nview_df.shape

In [ ]:
#remove currently irrelevant 'view' dummy values that are NaN

nview_df.drop(axis = 1, labels = {'view_EXCELLENT','view_FAIR', 'view_GOOD', 'view_NONE'}, inplace = True)
#verify columns removed
nview_df.info()

In [ ]:
#checking size of new dataframe
nview_df.shape

In [ ]:
#determining upper and lower price of these no view property
nview_df.price.describe()

In [ ]:
#plotting 'price' values of range_df
sns.boxplot(data = nview_df, x= 'price', color = "blue", palette = "rocket")

In [ ]:
#removing unwanted outliers
min_reach, max_reach = nview_df.price.quantile([0.05, 0.95])
min_reach, max_reach

In [ ]:
nview_df[nview_df.price > max_reach]

In [ ]:
nview_df[nview_df.price < min_reach]

In [ ]:
#creating new dataframe to represent view_NONE values only within our new price range
range_df = nview_df[(nview_df.price < max_reach) & (nview_df.price > min_reach)]
range_df

In [ ]:
range_df.shape

In [ ]:
#plotting 'price' values of range_df update
sns.boxplot(data = range_df, x= 'price', color = "blue", palette = "rocket")

In [ ]:
#reviewing for values of 0, removing these values and zips at this time
range_df.sum()

In [ ]:
#create new df removing currently irrelevant dummy, 'zips' columns
clean_df = range_df.drop(axis = 1, labels = {'zips','grade_12_Luxury', 'grade_13_Mansion', 'grade_2_Substandard'})
#verify columns removed
clean_df.info()

In [ ]:
preds_2 = clean_df.drop(labels = ['price'], axis = 1)
target_2 = clean_df.price

In [ ]:
X_2 = preds_2
y_2 = target_2

In [ ]:
model_2 = sm.OLS(y_2, sm.add_constant(X_2))
results_2 = model_2.fit()
print(results_2.summary())

### Our R-squared using is now 42.9%, F-statistic is below 0. We have mostly P-statistics above .5, but some are above this max. Looking at our values for 'sqft_lot', 'sqft_living' and 'sqft_above',  their P-statistics are below 0. So, we can move forward using those as our predictor values.

In [ ]:
# Check our current data's correlations with price
clean_df.price.corr(df.sqft_lot)

In [ ]:
clean_df.price.corr(df.sqft_living)

In [ ]:
clean_df.price.corr(df.sqft_above)

In [ ]:
sns.heatmap(X_2.corr())

### Final Model

## Regression Results

## The model represented is:
### Price = 812,600 + 0.3841(sqft_lot) + 96.8246(sqft_above) + 94.7702(sqft_living)

### Keeping in mind we are reviewing data only pertaining to original entries listed as view_NONE between the 95th and 5th percentiles of price:

#### Overall, this model is statistically significant with a t-statistic p-value and overall F_p-value still below 5%. 

##### This shows our sqft_lot, sqft_above and sqft_living parameters each significantly impact price
##### For each sf of increase in sqft_Lot we only gain .38 units in Price, even though it's p-value is still below 5%.
##### This shows us that sqft_lot is not a good fit for this linear regression model

In [ ]:
# This will model our chosen predictors alone without the effects of the other two
fig = plt.figure(figsize=(10,5))
sm.graphics.plot_partregress_grid(results_2, exog_idx=["sqft_lot", "sqft_above", "sqft_living"], fig=fig)
plt.tight_layout()
plt.show()

## Conclusion

### In conclusion, we can see that our initially chosen parameters sqft_lot, sqft_above, and sqft_living have some significance in determining final home selling price. With that said sqft_lot for this particular set of data, does not seem to add much to the price. However, their may be other factors that we may want to consider. We need to discern if the view values of 'NONE' are accurate, really NaN values, or are misleading in some other fashion. Other considerations may be the factors of condition and grade as they have p-values below 5% as well, we also see a larger stastistically significant impact on sales price of these homes. 

### Thank you,
### Scharmaine Chappell